In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import tensorflow.keras.backend as kb
from libs.feature_extraction import *

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [4]:
from matplotlib import font_manager, rc

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font',family=font_name)

plt.style.use('fivethirtyeight')
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (20,10)

In [5]:
df = get_df('KOSPI', 'KOSPI-All', 'Add')
df

,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,low,volume,대비%,한국금리,천연가스,미국금리,다우존스,나스닥,구리,WTI
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,1150.41,352670000,-2.05%,2.5,3.793,0.25,7223.98,1404.02,2.0370,67.16
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,1133.94,381510000,0.95%,2.5,3.550,0.25,7216.97,1462.11,2.0350,68.05
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,1151.80,358880000,1.28%,2.5,3.695,0.25,7395.70,1491.22,2.0280,68.38
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,1108.15,366480000,-6.03%,2.5,3.604,0.25,7486.58,1483.48,2.0750,67.23
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,1109.69,335690000,2.15%,2.5,3.535,0.25,7400.80,1457.27,2.0700,63.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,2993.46,1560000000,-2.45%,0.5,2.777,0.25,31402.01,13119.43,4.2635,63.53
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,3026.47,1280000000,3.50%,0.5,2.771,0.25,30932.37,13192.34,4.0925,61.50
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,2988.28,1360000000,-2.80%,0.5,2.777,0.25,31535.51,13588.83,4.1130,60.64


In [6]:
df = Feature_Extraction(df)
df

Loading...
Finish Moving Average
Loading...
Finish Volatility
Loading...
Finish Volume
Loading...
Finish Momentum
Loading...
Finish All
Number of Features: 65
Number of Datas: 3001


,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,PVT,BollingerU,BollingerM,BollingerL,Typical,EMV,Mass,NVI,PVI,CV
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,3.526700e+08,0.000000,0.000,0.000000,1161.450000,0.000000,0.000000,1000.00,1000.00,0.000000
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,3.562847e+08,0.000000,0.000,0.000000,1156.563333,-114.497916,0.000000,1000.00,1167.71,0.000000
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,3.608856e+08,0.000000,0.000,0.000000,1173.000000,156.543246,0.000000,1182.68,1167.71,0.000000
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,3.387793e+08,0.000000,0.000,0.000000,1121.633333,-420.710216,0.000000,1182.68,1111.34,0.000000
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,3.459864e+08,1200.840819,1150.736,1100.631181,1126.893333,-31.411123,0.000000,1135.20,1111.34,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,9.537298e+08,3144.720606,3067.820,2990.919394,3026.830000,-139.795564,26.661155,2994.98,3070.09,-0.069874
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,9.984810e+08,3150.520565,3070.426,2990.331435,3075.320000,116.755109,26.357024,3099.69,3070.09,-0.026551
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,9.604235e+08,3132.221372,3051.492,2970.762628,3030.240000,-180.466360,26.067053,3099.69,3012.95,0.050881


In [7]:
df = df.drop(columns='대비%')
df['pred_price'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)
df

,close,대비,기관,외국인,개인,기타,금융투자,보험,투신,사모,...,BollingerU,BollingerM,BollingerL,Typical,EMV,Mass,NVI,PVI,CV,pred_price
date,,,,,,,,,,,,,,,,,,,,,
2009-01-12,1156.75,-24.21,-105369,-100171,207988,-3455,-15603,-24208,-163712,-2151,...,0.000000,0.000,0.000000,1161.450000,0.000000,0.000000,1000.00,1000.00,0.000000,1
2009-01-13,1167.71,10.96,-262362,30223,205073,26175,-2955,4193,-253006,-16493,...,0.000000,0.000,0.000000,1156.563333,-114.497916,0.000000,1000.00,1167.71,0.000000,1
2009-01-14,1182.68,14.97,-5903,9837,-6527,2182,34413,-11807,-28832,-2658,...,0.000000,0.000,0.000000,1173.000000,156.543246,0.000000,1182.68,1167.71,0.000000,0
2009-01-15,1111.34,-71.34,-406404,-182570,563538,21116,-66342,31493,-439477,-5603,...,0.000000,0.000,0.000000,1121.633333,-420.710216,0.000000,1182.68,1111.34,0.000000,1
2009-01-16,1135.20,23.86,45743,-307461,236397,25424,5946,-7465,74338,8534,...,1200.840819,1150.736,1100.631181,1126.893333,-31.411123,0.000000,1135.20,1111.34,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,2994.98,-75.11,-82457,631358,561382,-1115543,258837,-27013,-68615,-34963,...,3144.720606,3067.820,2990.919394,3026.830000,-139.795564,26.661155,2994.98,3070.09,-0.069874,1
2021-02-25,3099.69,104.71,974913,974946,-1936090,-8948,1027147,-52027,5798,34583,...,3150.520565,3070.426,2990.331435,3075.320000,116.755109,26.357024,3099.69,3070.09,-0.026551,0
2021-02-26,3012.95,-86.74,-1030068,-2829962,3778506,68921,-891584,26706,-38506,-21671,...,3132.221372,3051.492,2970.762628,3030.240000,-180.466360,26.067053,3099.69,3012.95,0.050881,1


In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

### 지수X

In [30]:
feature_columns = df.columns.difference(["close"])

X = df[feature_columns]
y = df.pred_price

In [31]:
X.shape, y.shape

((3001, 64), (3001,))

In [18]:
rf = RandomForestClassifier(random_state = 0, class_weight="balanced")
rfe = RFE(estimator=rf, step=1)
X_new = rfe.fit_transform(X, y)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y,train_size=0.7)
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
param_grid = {
                 'n_estimators': [5, 10, 15, 20],
                 'max_depth': [2, 5, 7, 9]
             }
grid_clf = GridSearchCV(rf, param_grid, cv=k_fold.split(X_train, y_train))
grid_clf.fit(X_train, y_train)
y_pred = grid_clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[426,   0],
       [  0, 475]], dtype=int64)

In [23]:
y_pred, y_test

(array([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 

In [25]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX = scaler.fit_transform(X)

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)        

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)

0.525965379494008

In [35]:
dir(grid_search)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_validate_data',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'inverse_transform',
 'multimetric_',
 'n_features_in_',
 'n_jobs',
 'n_splits

In [40]:
grid_search.

{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}